### Function scopes in python

```python
def intersect(seq1, seq2):
res = []
for x in seq1:
    if x in seq2:
        res.append(x)
return res
```

The variable inside `intersect` is what is called a local variable-a name that is visible only to code inside the function def and exists only while the function runs. In fact, all names assigned in any way inside a function are classified as local variables by default, nearly all the names in intersect are local variables:

All these local variables appear when the function is called and disappear when the function exits-the return statement at the end of intersect sends back the result object but the name res goes away

Besides packaging code for reuse, functions add an extra namespace layer to your programs to minimize the potential for collisions among variables of the same name-by default, all names assigned inside a function are associated with that functions namespace, and no other. This rule means that:

- Names assigned inside a def can only be seen by the code within that def. You cannot even refer to such names from outside the function 
- Names assigned inside a def do not clash with variables outside the def, even if the same names are used elsewhere. A name X assigned outside a given def is a completely different variable from a name X assigned inside that def

`global` and `local` statements for variable declaration declare before variable assignment

- global makes scope lookup begin in the enclosing modules scope and allows names there to be assigned. Scope lookup continues on to the built-in scope if the name does not exist in the module, but assignments to global names always create or change them in the module's scope.

- nonlocal restricts scope lookup to just enclosing defs, requires that the names already exist there, and allows them to be assigned. Scope lookup does not continue on to the global or built-in scopes. 





### Function headers

Arbitrary arguments `*` and `**` are used to support functions that take any number of arguments. Both can appear in either the function definition or a function call, and they have related purporses in the two locations.

```python

def f(*args):
    print(args)

```

When this funcion is called, Python collects all the positional arguments into a new tuple and assigns the variable args to that tuple. Because it is a normal tuple object, it can be indexed, stepped through with a for loop and so on:

```python
f()
#returns () empty tuple
f(1)
#returns (1,)
f(1,2,3,4)
#returns (1,2,3,4)
```

The `**` feature is similar but it only works for keyword arguments-it collects them into a new dictionary which can then be processed with a normal dictionary tools. In a sense, th ** form allows you to convert keywords to dictionaries, which you can then step through with keys calls, dictionary iterators, and teh like

```python

def f(**args):
    print(args)

f()
# prints {}
f(a=1, b=2)
# prints {'a':1,'b':2}

```

Finally function headers can combine normal arguments, the * , and the ** to implement wildly flexible call signatures. For instance, in the following, 1 is passed to a by position, 2 and 3 are collected into the pargs positional tuple, and x and y wind up in the kargs keyword dictionary:

```python
def f(a, *pargs, **kwargs):
    print(a, pargs, kwargs)

f(1,2,3,x=1,y=2)
# prints 1 (2,3) {'y':2,'x':1}
```

Such code is rare but shows up in functions that need to support multiple call patterns (for backward compatibility, for instance). In fact these features can be combined in even more complex ways that may seem ambigous at first glance. 





### Generator Functions and Expressions

Two language constructs delay result creation whenever possible in user-defined operations:

- Generator functions are coded as normal def statements, but use `yield` statements to return results one at a time, suspending and resuming their state between each

- Generator expressions are similar to the list comprehensions but they return an object that produces results on demand instead of building a result list




In [11]:
def gensquares(N):
    for i in range(N):
        yield i ** 2 


The function yields a value, and so returns to its caller, each time through the loop; when it is resumed, its prior state is restored, including the last values of its variables i and N, and control picks up again immediately after the yield statement. For example, when its used in the body of a for loop, the first iteration starts the function and gets its first result, thereafter, control returns to the function after its yield statement each time through the loop:


In [12]:
for i in gensquares(5):
    print(i , end = ":")

0:1:4:9:16:

To end the generation of values, functions either use a return statement with no value or simply allow control to fall off the end of the function body.



### Generator expressions



In [13]:
(x ** 2 for x in range(4))

<generator object <genexpr> at 0x00000259719930C0>

In [19]:
for num in (x ** 2 for x in range(4)):
    print('%s, %s'% (num, num / 2.0))

0, 0.0
1, 0.5
4, 2.0
9, 4.5


The following deploys generator expressions in the string join method call and tuple assignment, iteration contexts both. In the first test here, join runs the generator and joins the substrings it produces with nothing between-to simply concatenate:


In [20]:
''.join(x.upper() for x in 'aaa,bbb,ccc'.split(','))

'AAABBBCCC'

Syntactically, parentheses are not required around a generator expression that is the sole item already enclosed in parentheses used for other purposes-like those of a function call. Parentheses are required in all other cases, however even if they seem extra as in the second call to sorted that follows:

In [21]:
sum(x ** 2 for x in range(4))

14

In [22]:
sorted(x ** 2 for x in range(4)) # this is optional 

[0, 1, 4, 9]

In [24]:
sorted((x ** 2 for x in range(4)), reverse=True) # the parens are required

[9, 4, 1, 0]

Just like generator functions, generator expressions are a memory space optimization they do not require the entire result list to be constructed all at once. 

In [40]:
%%time
x = 2
for i in range(100000000):
    x += 1

Wall time: 14.1 s


In [41]:
%%time
y = 2 
for i in trygen(100000000):
    y += i

Wall time: 21.5 s


Good uses of generator expression

In [42]:
list(map(lambda x: x * 2 , (1,2,3,4))) # non function case

[2, 4, 6, 8]

In [43]:
list(x * 2 for x in (1,2,3,4))# simpler as a generator

[2, 4, 6, 8]

In [44]:
line = 'aaa,bbb,ccc'
''.join([x.upper() for x in line.split(',')]) # this makes a pointless list

'AAABBBCCC'

In [50]:
''.join(x.upper() for x in line.split(',')) # generates results

'AAABBBCCC'

In [53]:
''.join(x * 2 for x in line.split(',')) # simpler as a generator

'aaaaaabbbbbbcccccc'

In [54]:
''.join(map(lambda x: x * 2, line.split(',')))

'aaaaaabbbbbbcccccc'

Although the effect of all these is to combine operations, the generators do so without making multiple temporary lists. This next example both nests and combines generators-the nested generator expression is activated by map, which in turn is only activated by lists

In [55]:
import math

In [58]:
list(map(math.sqrt, (x ** 2 for x in range(4))))

[0.0, 1.0, 2.0, 3.0]

In [61]:
# generator expressions like generator functions

G = (i ** 2 for i in range(4))
for i in G:
    print(i, end = ',')

0,1,4,9,

In [64]:
# or 
G = (i ** 2 for i in range(4))
print(list(G)) # same as in range, except that the generator only yields values once and its gone
print(list(G))

[0, 1, 4, 9]
[]


Temporary loop variable names in generator, set, dict, and list comprehensions are local to the expression. However the for loop iteration statement works differently


In [65]:
X = 99
[X for  X in range(5)]
X

99

In [66]:
Y = 99

for Y in range(4): # does not localize names
    pass
Y

3

In [37]:
def trygen(N):
    for i in range(N):
        yield i